In [1]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou

Dependencies:
torch: 0.4
torchvision
matplotlib
"""
# library
# standard library
import os

# third-party library
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

from torchvision.datasets import ImageFolder
import torch.utils.data as data
import torchvision
from torchvision import transforms
from torchvision import datasets

import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
def load_images(image_size=(28,28), batch_size=64, root="../datasets/MainImageFolder"):

    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_set = datasets.ImageFolder(root=root, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    return train_loader 

In [4]:
train_data = load_images(root='../../state-farm-distracted-driver-detection/imgs/train/')

In [15]:
valid_size = 0.2
batch_size = 64
num_workers = 0

train_ld = load_images(root='./train/')

train_data = train_ld.dataset

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)

valset= valid_loader.dataset

In [13]:
# batch_size = 64
# train_ld = load_images(root='./train/')
# train_ld1 = load_images(root='./train/')

# print(len(train_ld.dataset))
# validation_split = 0.3
# shuffle_dataset = True
# random_seed= 42

# # Creating data indices for training and validation splits:
# dataset_size = len(train_ld.dataset)
# indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
# train_idx, valid_idx = indices[split:], indices[:split]

# def get_same_index(target, label):
#     label_indices = []
    
#     for i in range(len(target)):
#         if target[i][1] == label:
#             label_indices.append(i)

#     return label_indices

# ind = get_same_index(train_ld.dataset.samples,0)
# sampler_chk1 = torch.utils.data.sampler.SubsetRandomSampler(ind)

# print(len(sampler_chk1))

# train_idx1 = list(set(train_idx) - set(sampler_chk1))
# print(len(train_idx1))

# valid_idx1 = list(set(valid_idx) - set(sampler_chk1))
# len(valid_idx1)

# train_data = train_ld.dataset
# train_data1 = train_ld1.dataset

# train_sampler = SubsetRandomSampler(train_idx1)
# valid_sampler = SubsetRandomSampler(valid_idx1)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#     sampler=train_sampler)
# print(len(train_loader.sampler))
# valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#     sampler=valid_sampler)
# print(len(valid_loader.sampler))
# valid_loader_ooc = torch.utils.data.DataLoader(train_data1, batch_size=batch_size, 
#     sampler=sampler_chk1)
# print(len(valid_loader_ooc.sampler))

# train_data=train_loader.dataset
# valid_data=valid_loader.dataset
# valset=valid_loader.dataset

22424
2489
13913
13913
6022
2489


In [16]:
# Insipired on LeNet, first proposed in: 
# https://ieeexplore.ieee.org/abstract/document/726791
# PDF: http://www.cs.virginia.edu/~vicente/deeplearning/readings/lecun1998.pdf
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # Convolutional layers.
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        # Linear layers.
        self.fc1 = nn.Linear(64*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Conv1 + ReLU + MaxPooling.
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        
        # Conv2 + ReLU + MaPooling.
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        
        # This flattens the output of the previous layer into a vector.
        out = out.view(out.size(0), -1) 
        
        # Linear layer + ReLU.
        out = F.relu(self.fc1(out))
        # Linear layer + ReLU.
        out = F.relu(self.fc2(out))
        # A final linear layer at the end.
        out = self.fc3(out)
       
        # We will not add Softmax here because nn.CrossEntropy does it.
        # Read the documentation for nn.CrossEntropy.
        return out

model = LeNet()

model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
correct = 0.0
cum_loss = 0.0

learningRate = 5e-2
optimizer = optim.SGD(model.parameters(), lr = learningRate, 
                      momentum = 0.9, weight_decay = 1e-4)
N = 5
# log accuracies and losses.
train_accuracies = []; val_accuracies = []
train_losses = []; val_losses = []
batchSize=64

In [47]:
for epoch in range(0, N):
        correct = 0.0
        cum_loss = 0.0

      # Make a pass over the training data.
        model.train()
        total_train=0
        for (i, (inputs, labels)) in enumerate(train_loader):
            inputs = inputs.cuda()
#             print(inputs.shape)
            labels = labels.cuda()
            total_train+=len(inputs)
            # Forward pass. (Prediction stage)
            scores = model(inputs)
            loss = loss_fn(scores, labels)

            # Count how many correct in this batch.
            max_scores, max_labels = scores.max(1)
            correct += (max_labels == labels).sum().item()
            cum_loss += loss.item()

            # Zero the gradients in the network.
            optimizer.zero_grad()

            # Backward pass. (Gradient computation stage)
            loss.backward()

            # Parameter updates (SGD step) -- if done with torch.optim!
            optimizer.step()

            # Parameter updates (SGD step) -- if done manually!
            # for param in model.parameters():
            #   param.data.add_(-learningRate, param.grad)

            # Logging the current results on training.
            if (i + 1) % 100 == 0:
                print('Train-epoch %d. Iteration %05d, Avg-Loss: %.4f, Accuracy: %.4f' % (epoch, i + 1, cum_loss / (i + 1), correct / ((i + 1) * batchSize)))

        train_accuracies.append(correct / total_train)
        train_losses.append(cum_loss / (i + 1))   

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        model.eval()
        total=0
        for (i, (inputs, labels)) in enumerate(valid_loader):
            inputs = inputs.cuda()
            labels = labels.cuda()
            
            total += len(inputs)
            # Forward pass. (Prediction stage)
            scores = model(inputs)
            cum_loss += loss_fn(scores, labels).item()

            # Count how many correct in this batch.
            max_scores, max_labels = scores.max(1)
            correct += (max_labels == labels).sum().item()

        val_accuracies.append(correct / total)
        val_losses.append(cum_loss / (i + 1))

        # Logging the current results on validation.
        print('Validation-epoch %d. Avg-Loss: %.4f, Accuracy: %.4f' % 
            (epoch, cum_loss / (i + 1), correct / total))

Train-epoch 0. Iteration 00100, Avg-Loss: 0.0574, Accuracy: 0.9830
Train-epoch 0. Iteration 00200, Avg-Loss: 0.0558, Accuracy: 0.9834
Validation-epoch 0. Avg-Loss: 0.1105, Accuracy: 0.9672
Train-epoch 1. Iteration 00100, Avg-Loss: 0.0846, Accuracy: 0.9736
Train-epoch 1. Iteration 00200, Avg-Loss: 0.0881, Accuracy: 0.9739
Validation-epoch 1. Avg-Loss: 0.0722, Accuracy: 0.9779
Train-epoch 2. Iteration 00100, Avg-Loss: 0.0225, Accuracy: 0.9936
Train-epoch 2. Iteration 00200, Avg-Loss: 0.0280, Accuracy: 0.9918
Validation-epoch 2. Avg-Loss: 0.0768, Accuracy: 0.9810
Train-epoch 3. Iteration 00100, Avg-Loss: 0.0232, Accuracy: 0.9925
Train-epoch 3. Iteration 00200, Avg-Loss: 0.0369, Accuracy: 0.9884
Validation-epoch 3. Avg-Loss: 0.1520, Accuracy: 0.9636
Train-epoch 4. Iteration 00100, Avg-Loss: 0.0321, Accuracy: 0.9898
Train-epoch 4. Iteration 00200, Avg-Loss: 0.0333, Accuracy: 0.9893
Validation-epoch 4. Avg-Loss: 0.0725, Accuracy: 0.9806


In [46]:
# Make a pass over the validation data.
correct = 0.0
cum_loss = 0.0
model.eval()
total=0
for (i, (inputs, labels)) in enumerate(valid_loader):
    inputs = inputs.cuda()
    labels = labels.cuda()

    # Forward pass. (Prediction stage)
    scores = model(inputs)
    cum_loss += loss_fn(scores, labels).item()

    # Count how many correct in this batch.
    max_scores, max_labels = scores.max(1)
    correct += (max_labels == labels).sum().item()
    total+= len(inputs)
    print(correct, total)

print(correct, len(valid_data))

val_accuracies.append(correct/total)
val_losses.append(cum_loss / (i + 1))

# Logging the current results on validation.
print('Validation-epoch %d. Avg-Loss: %.4f, Accuracy: %.4f' % 
    (epoch, cum_loss / (i + 1), correct/total ))

61.0 64
123.0 128
185.0 192
247.0 256
310.0 320
373.0 384
432.0 448
493.0 512
556.0 576
619.0 640
680.0 704
742.0 768
804.0 832
863.0 896
925.0 960
988.0 1024
1051.0 1088
1114.0 1152
1174.0 1216
1236.0 1280
1300.0 1344
1363.0 1408
1425.0 1472
1485.0 1536
1547.0 1600
1606.0 1664
1667.0 1728
1730.0 1792
1791.0 1856
1855.0 1920
1918.0 1984
1982.0 2048
2044.0 2112
2108.0 2176
2171.0 2240
2234.0 2304
2297.0 2368
2361.0 2432
2424.0 2496
2487.0 2560
2551.0 2624
2612.0 2688
2676.0 2752
2740.0 2816
2804.0 2880
2865.0 2944
2928.0 3008
2990.0 3072
3053.0 3136
3116.0 3200
3177.0 3264
3237.0 3328
3300.0 3392
3364.0 3456
3424.0 3520
3486.0 3584
3548.0 3648
3609.0 3712
3672.0 3776
3734.0 3840
3796.0 3904
3857.0 3968
3917.0 4032
3980.0 4096
4042.0 4160
4104.0 4224
4167.0 4288
4229.0 4352
4291.0 4416
4354.0 4480
4358.0 4484
4358.0 22424
Validation-epoch 4. Avg-Loss: 0.0969, Accuracy: 0.9719


In [40]:
# len(valid_loader.sampler)
valid_loader.batch_size*len(valid_loader)
valid_loader.batch_sampler.batch_size

64